# (1st part) DATA PREPROCESSING
                                                 BY MAHENDRA NANDI

- This is the 1st notebook ( or 1st part of the project, we have two different notebook, one is for preprocessin g and other is for model and training) of the two.
- Import libraries if needed.
- follow the codes carefully, you can get why that has been done before something else. You need to understand the 

- The almost same code is present in the py files out side this folder.If you are done with practicing in notebook you may then go through the py files.
- Ignore first 6 cells. These are for some other reasons which you can get after going through the whole notebook.

In [ ]:
%pycat preprocessing.py

In [71]:
%cd /content/drive/MyDrive/BDA2020_MN/Projects/My_Projects/

/content/drive/MyDrive/BDA2020_MN/Projects/My_Projects
time: 15.6 ms (started: 2021-06-18 08:56:39 +00:00)


In [72]:
ls

'Copy of image_captioning_preprocessing.ipynb'
 demo.mahi
 image_captioning.ipynb
 image_captioning_preprocessing.ipynb
 image_captioning_training_testing_12th_june_55epochs.ipynb
 image_processing.py
 preprocessing.py
 pypy1.py
 text_processing.py
time: 125 ms (started: 2021-06-18 08:56:42 +00:00)


In [82]:
!python image_processing.py

2021-06-18 09:48:00.106854: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
time: 2.85 s (started: 2021-06-18 09:47:59 +00:00)


In [78]:
!python text_processing.py

2021-06-18 09:27:35.541778: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
time: 3.04 s (started: 2021-06-18 09:27:34 +00:00)


# Image Captioning using LSTM on Flickr8K dataset

## Downloading Data and Dependencies

In [52]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.75 s (started: 2021-06-18 08:49:45 +00:00)


In [53]:
#importing the libraries
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input        # check if require
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical
from keras.utils import np_utils
from nltk.translate.bleu_score import sentence_bleu
from numpy import array
import pickle
import numpy as np
import os
import cv2

time: 11.1 ms (started: 2021-06-18 08:49:48 +00:00)


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.57 ms (started: 2021-06-18 08:49:48 +00:00)


## Data Visualization and Preprocessing

In [55]:
#converting the text files to pandas dataframe
image_tokens=pd.read_csv("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/Flickr8k_text/Flickr8k.lemma.token.txt",sep='\t',names=["img_id","img_caption"])
train_image_names=pd.read_csv("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/Flickr8k_text/Flickr_8k.trainImages.txt",names=["img_id"])
test_image_names=pd.read_csv("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/Flickr8k_text/Flickr_8k.testImages.txt",names=["img_id"])
val_image_names=pd.read_csv("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/Flickr8k_text/Flickr_8k.devImages.txt",names=["img_id"])

time: 114 ms (started: 2021-06-18 08:49:48 +00:00)


In [56]:
train_image_names.img_id[0:2]

0    2513260012_03d33305cf.jpg
1    2903617548_d3e38d7f88.jpg
Name: img_id, dtype: object

time: 11.6 ms (started: 2021-06-18 08:49:48 +00:00)


In [57]:
#removing the #0,#1,#2,#3,#5 from the image ids
image_tokens["img_id"]=image_tokens["img_id"].map(lambda x: x[:len(x)-2])
image_tokens["img_caption"]=image_tokens["img_caption"].map(lambda x: "<start> " + x.strip() + " <end>")

time: 45.6 ms (started: 2021-06-18 08:49:48 +00:00)


In [58]:
import numpy as np
import random
a=random.shuffle([i for i in range(10)])
a

time: 2.58 ms (started: 2021-06-18 08:49:48 +00:00)


### Creating dictionaries to map image_id and their corresponding captions

In [59]:
#creating train dictionary having key as the image id and value as a list of its captions
train_captions={}
for i in tqdm(range(len(train_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==train_image_names["img_id"].iloc[i]].img_caption)]
  train_captions[train_image_names["img_id"].iloc[i]]=l

100%|██████████| 6000/6000 [00:16<00:00, 359.89it/s]

time: 16.7 s (started: 2021-06-18 08:49:48 +00:00)


In [60]:
#creating test dictionary having key as the image id and value as a list of its captions
test_captions={}
for i in tqdm(range(len(test_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==test_image_names["img_id"].iloc[i]].img_caption)]
  test_captions[test_image_names["img_id"].iloc[i]]=l

100%|██████████| 1000/1000 [00:02<00:00, 357.82it/s]

time: 2.81 s (started: 2021-06-18 08:50:05 +00:00)


In [61]:
#creating validation dictionary having key as the image id and value as a list of its captions
validation_captions={}
for i in tqdm(range(len(val_image_names))):
  l=[caption for caption in(image_tokens[image_tokens["img_id"]==val_image_names["img_id"].iloc[i]].img_caption)]
  validation_captions[val_image_names["img_id"].iloc[i]]=l
# validation_captions

100%|██████████| 1000/1000 [00:02<00:00, 351.25it/s]

time: 2.86 s (started: 2021-06-18 08:50:08 +00:00)


In [62]:

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/train_captions.pickle","wb")
# pickle.dump(train_captions,pickle_out)
# pickle_out.close()

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/test_captions.pickle","wb")
# pickle.dump(test_captions,pickle_out)
# pickle_out.close()

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/validation_captions.pickle","wb")
# pickle.dump(validation_captions,pickle_out)
# pickle_out.close()

time: 3.85 ms (started: 2021-06-18 08:50:11 +00:00)


### Setting hyper parameters for vocabulary size and maximum length

In [63]:
all_captions=[]
for img_id in tqdm(train_captions):
  for captions in train_captions[img_id]:
    all_captions.append(captions)

all_words=" ".join(all_captions)
print()
print(len(all_words))
unique_words=list(sorted(set(all_words.strip().split(" "))))
print(len(unique_words))


100%|██████████| 6000/6000 [00:00<00:00, 228120.74it/s]



1991192
6328
time: 119 ms (started: 2021-06-18 08:50:11 +00:00)


In [64]:
#defining max_length and vocabulary size
vocab_size=len(unique_words)+1
max_length=40

time: 8.34 ms (started: 2021-06-18 08:50:11 +00:00)


### Creating dictionaries containg mapping of words to indices and indices to words

In [40]:
#forming dictionaries containg mapping of words to indices and indices to words
words_to_indices={val:index+1 for index, val in enumerate(unique_words)}
indices_to_words = { index+1:val for index, val in enumerate(unique_words)}
words_to_indices["Unk"]=0
indices_to_words[0]="Unk"

time: 10.6 ms (started: 2021-06-18 07:04:46 +00:00)


### Transforming data into dictonary mapping of image_id to encoded captions

In [43]:
# #forming dictionary having encoded captions
# max_length=40
# train_encoded_captions={}
# for img_id in tqdm(train_captions):
#   train_encoded_captions[img_id]=[]
#   for i in range(5):
#     train_encoded_captions[img_id].append([words_to_indices[s] for s in train_captions[img_id][i].split(" ")])

# for img_id in tqdm(train_encoded_captions):
#   train_encoded_captions[img_id]=pad_sequences(train_encoded_captions[img_id], maxlen=max_length, padding='post')

100%|██████████| 6000/6000 [00:00<00:00, 26297.79it/s]

time: 515 ms (started: 2021-06-18 07:34:34 +00:00)


In [44]:
# #forming dictionary having encoded captions
# test_encoded_captions={}
# for img_id in tqdm(test_captions):
#   test_encoded_captions[img_id]=[]
#   for i in range(5):
#     test_encoded_captions[img_id].append([words_to_indices[s] for s in test_captions[img_id][i].split(" ") if s in words_to_indices ])

# for img_id in tqdm(test_encoded_captions):
#   test_encoded_captions[img_id]=pad_sequences(test_encoded_captions[img_id], maxlen=max_length, padding='post')

100%|██████████| 1000/1000 [00:00<00:00, 17758.25it/s]

time: 101 ms (started: 2021-06-18 07:34:41 +00:00)


In [45]:
# #forming dictionary having encoded captions
# validation_encoded_captions={}
# for img_id in tqdm(validation_captions):
#   validation_encoded_captions[img_id]=[]
#   for i in range(5):
#     validation_encoded_captions[img_id].append([words_to_indices[s] for s in validation_captions[img_id][i].split(" ") if s in words_to_indices ])

# for img_id in tqdm(validation_encoded_captions):
#   validation_encoded_captions[img_id]=pad_sequences(validation_encoded_captions[img_id], maxlen=max_length, padding='post')

100%|██████████| 1000/1000 [00:00<00:00, 29221.68it/s]

time: 75.4 ms (started: 2021-06-18 07:34:44 +00:00)


In [68]:
def encoding_captions(train_captions,test_captions,validation_captions,max_length ):
  TTV=[train_captions,test_captions,validation_captions]
  train_encoded_captions={}
  test_encoded_captions={}
  validation_encoded_captions={}
  encoded_dictionaries=[train_encoded_captions,test_encoded_captions,validation_encoded_captions]
  
  for j in range(3):
    ttv=TTV[j]
    encded_dict=encoded_dictionaries[j]
    
    for img_id in tqdm(ttv):
      encded_dict[img_id]=[]
      for i in range(5):
        encded_dict[img_id].append([words_to_indices[s] for s in ttv[img_id][i].split(" ") if s in words_to_indices ])
    
    for img_id in tqdm(encded_dict):
      encded_dict[img_id]=pad_sequences(encded_dict[img_id], maxlen=max_length, padding='post')
  return train_encoded_captions,test_encoded_captions,validation_encoded_captions

time: 17 ms (started: 2021-06-18 08:53:09 +00:00)


In [69]:
train_encoded_captions,test_encoded_captions,validation_encoded_captions=encoding_captions(train_captions,test_captions,validation_captions,max_length )

100%|██████████| 1000/1000 [00:00<00:00, 22062.40it/s]

time: 566 ms (started: 2021-06-18 08:53:44 +00:00)


In [ ]:

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/words_to_indices.pickle","wb")
# pickle.dump(words_to_indices,pickle_out)
# pickle_out.close()


# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/indices_to_words.pickle","wb")
# pickle.dump(indices_to_words,pickle_out)
# pickle_out.close()

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/train_encoded_captions.pickle","wb")
# pickle.dump(train_encoded_captions,pickle_out)
# pickle_out.close()

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/test_encoded_captions.pickle","wb")                                 # I did intensionally it later 
# pickle.dump(test_encoded_captions,pickle_out)
# pickle_out.close()

# pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/validation_encoded_captions.pickle","wb")                                 # I did intensionally it later 
# pickle.dump(validation_encoded_captions,pickle_out)
# pickle_out.close()

time: 3.49 ms (started: 2021-05-22 08:43:19 +00:00)


## ResNet50 model for encoding images

In [ ]:
from keras.applications.resnet50 import ResNet50

model=ResNet50(include_top=False, weights='imagenet',pooling='avg',input_shape=(224,224,3))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Encoding images and forming dictionaries containing mapping of image_id to image encodings

In [ ]:
#extracting image encodings(features) from resnet50 and forming dict train_features
# path="all_images/Flicker8k_Dataset/"
all_images="/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/Flickr8k_Dataset/Flicker8k_Dataset/"
path=all_images
train_features={}
c=0
for image_name in tqdm(train_captions):
  img_path=path+image_name
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  train_features[image_name]=features.squeeze()

# storing_train_features= train_features

100%|██████████| 6000/6000 [45:56<00:00,  2.18it/s]

time: 45min 56s (started: 2021-05-22 08:43:21 +00:00)


In [ ]:
#extracting image encodings(features) from resnet50 and forming dict test_features
# path="all_images/Flicker8k_Dataset/"
path=all_images
test_features={}
c=0
for image_name in tqdm(test_captions):
  img_path=path+image_name
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  test_features[image_name]=features.squeeze()

100%|██████████| 1000/1000 [07:47<00:00,  2.14it/s]

time: 7min 47s (started: 2021-05-22 09:29:17 +00:00)


In [ ]:
#extracting image encodings(features) from resnet50 and forming dict validation_features
# path="all_images/Flicker8k_Dataset/"
path=all_images
validation_features={}
c=0
for image_name in tqdm(validation_captions):
  img_path=path+image_name
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  validation_features[image_name]=features.squeeze()

100%|██████████| 1000/1000 [07:39<00:00,  2.17it/s]

time: 7min 39s (started: 2021-05-22 09:37:05 +00:00)


In [ ]:
pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/train_features.pickle","wb")
pickle.dump(train_features,pickle_out)
pickle_out.close()

pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/test_features.pickle","wb")
pickle.dump(test_features,pickle_out)
pickle_out.close()

pickle_out= open("/content/drive/MyDrive/BDA2020_MN/MAHENDRA_DATASET/validation_features.pickle","wb")
pickle.dump(validation_features,pickle_out)
pickle_out.close()

time: 322 ms (started: 2021-05-22 09:44:45 +00:00)
